In [ ]:
%%capture
%pip install speechbrain
%pip install transformers

In [ ]:
from speechbrain.pretrained import SepformerSeparation as separator
import torchaudio
from speechbrain.dataio.dataio import read_audio
from IPython.display import Audio

from speechbrain.pretrained.interfaces import fetch
from speechbrain.pretrained.interfaces import SNREstimator as snrest

In [ ]:
model = separator.from_hparams(source="speechbrain/sepformer-wham-enhancement", savedir='pretrained_models/sepformer-wham-enhancement')


Downloading:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

In [ ]:
est_sources = model.separate_file(path='speechbrain/sepformer-wham-enhancement/example_wham.wav')

In [ ]:
signal = read_audio("example_wham.wav").squeeze()
batch = 1.5
Audio(signal, rate=8000)

In [ ]:
torchaudio.save("enhanced_wham.wav", est_sources[:, :, 0].detach().cpu(), 8000)
signal = read_audio("enhanced_wham.wav").squeeze()
Audio(signal, rate=8000)

In [ ]:
model = separator.from_hparams(source="speechbrain/sepformer-wham", savedir='pretrained_models/sepformer-wham')
est_sources = model.separate_file(path='example_wham.wav')

# 3- Estimate the performance
snr_est_model = snrest.from_hparams(source="speechbrain/REAL-M-sisnr-estimator",savedir='pretrained_models/REAL-M-sisnr-estimator')
mix, fs = torchaudio.load('enhanced_wham.wav')
snrhat = snr_est_model.estimate_batch(mix, est_sources)
print(snrhat*batch)

tensor([1.4996e+01, 2.5670e-05])


In [ ]:
from google.colab import output
from PIL import Image
from io import BytesIO
import time

def take_screenshot():
    output.eval_js('new ResizeObserver(entries => {const entry = entries[0]; if (entry.target.clientHeight !== entry.contentRect.height) {entry.target.style.height = entry.contentRect.height + "px";}}).observe(document.querySelector("#output-container"))')
    time.sleep(1)  # add a delay to give the page time to render
    image = output.eval_js('new ImageData(0, 0)')
    buffer = BytesIO()
    image.save(buffer, format='PNG')
    buffer.seek(0)
    img = Image.open(buffer)
    return img


In [ ]:
take_screenshot()